In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import os
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
#import matplotlib
# matplotlib.use('Agg')
import lightgbm as lgb
from sklearn import metrics
from numpy.ma import MaskedArray
import sklearn.utils.fixes
sklearn.utils.fixes.MaskedArray = MaskedArray
from sklearn.model_selection import train_test_split
#from sklearn.metrics import plot_confusion_matrix
import shap

In [8]:
# **************** Loading the data
# For the example I am usign a table in csv format, but if your data are larger you can use other formats compatible with pandas, like .pickle
print("Start loading data file ")

cols_names=['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type']
input_file='phia0.2/particles-features-0.2-Fa100.txt'
data = pd.read_csv(input_file, delimiter=' ',names=cols_names)

#print(data)


# *** Split the available data in training and test
# the model will be trained only using the training data
# so we can evaluate the performance on a different (test) set, that is new for the ML model
df_train, df_test = train_test_split(data, random_state=50, test_size=0.3)

print('------------ BEGIN TRAIN DATAFRAME COLUMNS ------------------')
print(df_train.columns)
print('------------- END TRAIN DATAFRAME COLUMNS -------------------')


Start loading data file 
------------ BEGIN TRAIN DATAFRAME COLUMNS ------------------
Index(['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type'],
      dtype='object')
------------- END TRAIN DATAFRAME COLUMNS -------------------


In [9]:
df_train

,area,perimeter,neighbours,max neighbour distance,min neighbour distance,max vertices distance,min vertices distance,max vertices-point distance,min vertices-point distance,distance to center,activity,particle type
256079,1.047011,4.178238,7.0,2.201113,0.859966,1.585309,0.074016,1.115118,0.509757,0.200030,1.0,1.0
223616,0.843462,3.576157,6.0,1.383122,0.763237,1.295792,0.288507,0.770265,0.484701,0.056127,0.0,1.0
106357,1.443148,4.596134,7.0,1.884285,0.809504,1.591790,0.141635,0.981149,0.527731,0.232712,0.0,1.0
249231,0.793610,3.328668,6.0,1.025938,0.854045,1.128944,0.456989,0.580893,0.502361,0.027720,0.0,1.0
62589,0.723417,3.261669,6.0,1.084644,0.777992,1.099648,0.221656,0.593483,0.516046,0.014075,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
385156,0.853660,3.547469,7.0,1.573611,0.787837,1.333288,0.062805,0.802532,0.500713,0.125366,1.0,0.0
321502,0.824707,3.457657,6.0,1.173055,0.824915,1.247192,0.238863,0.670915,0.523636,0.039653,0.0,1.0
153709,0.852645,3.520109,7.0,1.324826,0.930400,1.200821,0.033387,0.664345,0.557557,0.028646,0.0,1.0
239499,0.664348,3.064091,6.0,0.988707,0.754418,1.104906,0.340833,0.553945,0.490128,0.009155,0.0,1.0


In [10]:
df_test

,area,perimeter,neighbours,max neighbour distance,min neighbour distance,max vertices distance,min vertices distance,max vertices-point distance,min vertices-point distance,distance to center,activity,particle type
356285,0.862720,3.560365,6.0,1.282138,0.850024,1.237910,0.266768,0.668870,0.537921,0.057779,0.0,1.0
208432,0.697077,3.147268,6.0,1.067320,0.808950,1.064000,0.324711,0.578226,0.485662,0.042624,0.0,1.0
176817,0.607334,2.989735,5.0,0.898130,0.750549,1.055932,0.490923,0.541111,0.470767,0.003823,0.0,0.0
61545,0.951954,3.834312,6.0,1.450819,0.846600,1.302368,0.192530,0.742805,0.560335,0.043711,0.0,1.0
251567,0.851699,3.568114,6.0,1.226069,0.789624,1.239907,0.196848,0.649284,0.592010,0.020866,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
31257,0.980678,3.840923,6.0,1.354348,0.828374,1.341337,0.111404,0.738692,0.559965,0.086522,0.0,1.0
339003,0.924079,3.682522,6.0,1.419942,0.804500,1.277137,0.178859,0.734591,0.513996,0.110695,1.0,1.0
177187,2.005321,5.652876,7.0,2.385760,0.877376,2.290972,0.208174,1.588113,0.688401,0.394747,1.0,0.0
212221,3.042992,6.447470,9.0,3.077431,1.091575,2.300149,0.166270,1.621860,0.699589,0.463463,0.0,1.0


In [11]:
print("create x/y dataframes (train set)")
# In this particulat proble we want to identify if a specific particle 'is_active' or not
# so we have to do a binary classification of the column 'is_active'
df_train_y = df_train['activity'].copy().astype('int')
df_train.drop(columns='activity', inplace=True)
df_train_x = df_train

print("create x/y dataframes (test set)")
df_test_y = df_test['activity'].copy().astype('int')
df_test.drop(columns='activity', inplace=True)
df_test_x = df_test

create x/y dataframes (train set)
create x/y dataframes (test set)


In [12]:
print('\n\n*** Using a gradient boosting algorithm')
# This is a more advanced algorithm that is explained here:https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
model = lgb.LGBMClassifier(learning_rate=0.01,num_leaves=100,max_depth=15, early_stopping_rounds=10, num_iterations=3000, random_state=42)

model.fit(df_train_x,df_train_y,eval_set=[(df_train_x,df_train_y),(df_test_x,df_test_y)],
            eval_metric='binary')
print('\n\n***Training accuracy {:.4f}'.format(model.score(df_train_x,df_train_y)))
print('***Testing accuracy {:.4f}\n'.format(model.score(df_test_x,df_test_y)))

print(metrics.classification_report(df_test_y,model.predict(df_test_x)))



*** Using a gradient boosting algorithm
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 56159, number of negative: 224541
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 280700, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200068 -> initscore=-1.385871
[LightGBM] [Info] Start training from score -1.385871
Training until validation scores d

c:\Users\Pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[1044]	training's binary_logloss: 0.387569	valid_1's binary_logloss: 0.411497


***Training accuracy 0.8373
***Testing accuracy 0.8283

              precision    recall  f1-score   support

           0       0.84      0.97      0.90     96259
           1       0.69      0.26      0.37     24041

    accuracy                           0.83    120300
   macro avg       0.76      0.61      0.64    120300
weighted avg       0.81      0.83      0.79    120300



In [6]:
pred_test=model.predict(df_test_x)
from sklearn.metrics import roc_auc_score
roc_auc_score(df_test_y,pred_test)

0.5950115028085902

In [7]:
f1_score(df_test_y,pred_test)

0.3100538413750259

In [23]:
#Returns accuracy,auc,f1_score
def score(y,pred):
    return accuracy_score(y,pred),roc_auc_score(y,pred),f1_score(y,pred)

In [22]:
df=pd.read_csv("phia0.1/traj_phia0.1-T05-Fa20-tau1.dat")
df 


,1 1 7.32901 24.9911
0,2 1 24.2163 28.0885
1,3 1 24.8592 2.49579
2,4 1 16.2195 8.41785
3,5 1 10.2535 24.3277
4,6 1 19.2187 23.6963
...,...
2000994,996 4 26.0632 4.15549
2000995,997 4 23.9477 22.5588
2000996,998 4 25.0032 23.802
2000997,999 4 16.861 17.2506


In [26]:
def gb_model(df_train_x,df_train_y,df_test_x,df_test_y):
    model = lgb.LGBMClassifier(learning_rate=0.01,num_leaves=100,max_depth=15, early_stopping_rounds=10, num_iterations=3000, random_state=42)

    model.fit(df_train_x,df_train_y,eval_set=[(df_train_x,df_train_y),(df_test_x,df_test_y)],
            eval_metric='binary')
    return model

In [30]:
def plot_score_Fa_depend(fa_list,density,model_method):
    cols_names=['area', 'perimeter', 'neighbours', 'max neighbour distance',
       'min neighbour distance', 'max vertices distance',
       'min vertices distance', 'max vertices-point distance',
       'min vertices-point distance', 'distance to center', 'activity',
       'particle type']
    accuracy_list=[]
    auc_list=[]
    f1_list=[]

    for fa in fa_list:
        input_file=f"phia{density}/particles-features-{density}-Fa{fa}.txt"
        data = pd.read_csv(input_file, delimiter=' ',names=cols_names)

        df_train, df_test = train_test_split(data, random_state=50, test_size=0.3)
        df_train_y = df_train['activity'].copy().astype('int')
        df_train.drop(columns='activity', inplace=True)
        df_train_x = df_train

        df_test_y = df_test['activity'].copy().astype('int')
        df_test.drop(columns='activity', inplace=True)
        df_test_x = df_test

        model=model_method(df_train_x,df_train_y,df_test_x,df_test_y)
        pred=model.predict(df_test_x)
        accuracy,auc,f1=score(df_test_y,pred)
        accuracy_list.append(accuracy)
        auc_list.append(auc)
        f1_list.append(f1)
    
    plt.scatter(fa_list,accuracy_list)
    plt.scatter(fa_list,auc_list)
    plt.scatter(fa_list,f1_list)
    plt.legend(['Accuracy','ROC AUC score', 'F1 score'])
    return accuracy_list,auc_list,f1_list
        

In [31]:
Fa_list=[100,85,75,60,40,20]
plot_score_Fa_depend(Fa_list,0.1,gb_model)

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 13991, number of negative: 126009
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2306
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.099936 -> initscore=-2.197939
[LightGBM] [Info] Start training from score -2.197939
Training unt

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4234, number of negative: 37766
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000986 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2305
[LightGBM] [Info] Number of data points in the train set: 42000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100810 -> initscore=-2.188262
[LightGBM] [Info] Start training from score -2.188262
Training until validation scores don't improve for 10 rounds
Early stopping, be

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 140097, number of negative: 1259903
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030301 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2305
[LightGBM] [Info] Number of data points in the train set: 1400000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100069 -> initscore=-2.196455
[LightGBM] [Info] Start training from score -2.196455
Training until validation scores don't improve for 10 rounds
Early stoppi

c:\Users\Pedro\Documentos\pruebatfg\myenvprueba\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 140115, number of negative: 1259885
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028570 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2304
[LightGBM] [Info] Number of data points in the train set: 1400000, number of used features: 11
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.100082 -> initscore=-2.196312
[LightGBM] [Info] Start training from score -2.196312
Training until validation scores don't improve for 10 rounds
Early stoppi

([0.91495,
  0.9122777777777777,
  0.9111533333333334,
  0.908095,
  0.9034833333333333,
  0.900395],
 [0.6266326880740732,
  0.5897841254551394,
  0.5950115028085902,
  0.5705903378676136,
  0.5340524729333177,
  0.5031788887870942],
 [0.3852547885796892,
  0.29665924276169264,
  0.3100538413750259,
  0.24437836578648067,
  0.1303499023877459,
  0.01363283764379673])

In [40]:
fa_list=[100,85,75,60,40,20]
accuracy_list=[0.91495,
  0.9122777777777777,
  0.9111533333333334,
  0.908095,
  0.9034833333333333,
  0.900395]
auc_list=[0.6266326880740732,
  0.5897841254551394,
  0.5950115028085902,
  0.5705903378676136,
  0.5340524729333177,
  0.5031788887870942]
f1_list=[0.3852547885796892,
  0.29665924276169264,
  0.3100538413750259,
  0.24437836578648067,
  0.1303499023877459,
  0.01363283764379673]
plt.scatter(fa_list,accuracy_list)
plt.scatter(fa_list,auc_list)
plt.scatter(fa_list,f1_list)
plt.legend(['Accuracy','ROC AUC score', 'F1 score'])
plt.show()


C:\Users\Pedro\AppData\Local\Temp\ipykernel_8296\1590880250.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
score(df_test_y,pred_test)

(0.908095, 0.5705903378676136, 0.24437836578648067)

In [ ]:
Fa=[100,85,75,60,40,20]
accuracy=[0.9045471428571429,0.900395]
auc_score=[0.5378830396854314,0.5031788887870942]
f1_score0=[0.95,0.95,0.95]
f1_score1=[0.24,0.13,0.01 ]



In [53]:
pred_test=model.predict(df_test_x)
print(f'The accuracy on the test set is {accuracy_score(df_test_y,pred_test)} and the AUC is {roc_auc_score(df_test_y,pred_test)}')

The accuracy on the test set is 0.908095 and the AUC is 0.5705903378676136


In [19]:
pred_base=np.zeros_like(pred_test)
print(accuracy_score(df_test_y,pred_base))
print(roc_auc_score(df_test_y,pred_base))

0.9001916666666666
0.5


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rforestmodel=RandomForestClassifier(n_jobs=-1)
rforestmodel.fit(df_train_x,df_train_y)
pred_test=rforestmodel.predict(df_test_x)

In [23]:
print(accuracy_score(df_test_y,pred_test))
print(roc_auc_score(df_test_y,pred_test))

0.9002816666666666
0.504177515095089


In [39]:
import seaborn as sns
for col in data.columns:
    sns.kdeplot(data[data['activity'] == 0][col], label="Pasivas", fill=True)
    sns.kdeplot(data[data['activity'] == 1][col], label="Activas", fill=True)
    plt.title(col)
    plt.legend()
    plt.savefig(f"{col}_distribution.png")  # Guarda la imagen
    plt.close()  # Cierra la figura para liberar memoria

C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:3: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data[data['activity'] == 0][col], label="Pasivas", fill=True)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:4: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(data[data['activity'] == 1][col], label="Activas", fill=True)
C:\Users\Pedro\AppData\Local\Temp\ipykernel_12060\3059332063.py:6: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [53]:
import joblib
density=0.1
fa=100
model,features,target,df_test_y,df_test_x=joblib.load(f"phia{density}/gb-model-{density}-Fa{fa}.pkl")

In [54]:
pred=model.predict(df_test_x)

In [55]:
accuracy_score(df_test_y,pred)

0.91495

In [56]:
roc_auc_score(df_test_y,pred)

0.6266326880740732